In [38]:
# Imports
suppressWarnings(suppressPackageStartupMessages({
    library(Seurat)
    library(DropletUtils)
    library(scCustomize)
    library(rhdf5)
}))

In [3]:
# Globals
prj.dir <- "/scratch/gpfs/KANG/sereno/spatialstem"
src.dir <- file.path(prj.dir, "sourcefiles")
raw.dir <- file.path(src.dir, "raw")
had.dir <- file.path(src.dir, "h5ad")
int.dir <- file.path(prj.dir, "intermediates")
fig.dir <- file.path(prj.dir, "fig")

In [72]:
# Function for exporting our seurat objects in a scanpy-readable fashion.
seurat_to_10x <- function(seur.obj, out.dir) {
    seur.data <- GetAssayData(seur.obj)
    write10xCounts(path=out.dir, x=seur.data, version="3")
    coords <- seur.obj@images[[1]]@coordinates[, c("row", "col")]
    coord.path <- file.path(out.dir, "coords.csv")
    write.table(coords, file=coord.path, sep=",", quote=FALSE, row.names=FALSE, col.names=FALSE)
}

In [51]:
# Larouche: Spatiotemporal mapping of immune and stem cell dysregulation after volumetric muscle loss
laro.dir <- file.path(raw.dir, "pub4_larouche")
mm.file <- file.path(laro.dir, "GSE205707_Mus_ST_list_label_transfer_1201.RDS")
ca.file <- file.path(laro.dir, "GSE205707_Canis_ST_list.RDS")

In [54]:
# Larouche: get mouse data and set labels
mm.seurs <- readRDS(mm.file)
mm.seurs
mm.labels <- c("PBS1", "ITD1", "ITD2", "PBS2", "PBS3", "PBS4", "ITD3", "ITD4", "D141", "D142")

$PBS_599L
An object of class Seurat 
47291 features across 721 samples within 3 assays 
Active assay: SCT (14982 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 2 other assays present: Spatial, predictions
 2 dimensional reductions calculated: pca, umap
 1 image present: PBS_599L

$ITD1_599R
An object of class Seurat 
47224 features across 866 samples within 3 assays 
Active assay: SCT (14915 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 2 other assays present: Spatial, predictions
 2 dimensional reductions calculated: pca, umap
 1 image present: ITD1_599R

$ITD1_600L
An object of class Seurat 
47646 features across 654 samples within 3 assays 
Active assay: SCT (15337 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 2 other assays present: Spatial, predictions
 2 dimensional reductions calculated: pca, umap
 1 image present: ITD1_600L

$PBS_600R
An object of class Seurat 
47284 features acros

In [83]:
# Larouche: save mouse data
for (idx in seq(1:length(mm.seurs))) {
    mm.seur <- mm.seurs[[idx]]
    mm.label <- paste0("mm", mm.labels[[idx]])
    out.dir <- file.path(laro.dir, mm.label)
    seurat_to_10x(mm.seur, out.dir)
}

In [86]:
# Larouche: get canine data and set labels
ca.seurs <- readRDS(ca.file)
ca.seurs
ca.labels <- c("D7M1", "D7E1", "D14M1", "D14E1", "D0IR1", "D0IR2", "D7M2", "D14M2")

$`Sample_5513-JL-S2-A_CGCGCACT-AGAATACA`
An object of class Seurat 
46272 features across 2888 samples within 2 assays 
Active assay: SCT (13568 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: Spatial
 2 dimensional reductions calculated: pca, umap
 1 image present: D7_Mid_1

$`Sample_5513-JL-S2-B_CCTGTCAG-GTTACGGG`
An object of class Seurat 
45087 features across 1704 samples within 2 assays 
Active assay: SCT (12383 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: Spatial
 2 dimensional reductions calculated: pca, umap
 1 image present: D7_Edge_1

$`Sample_5513-JL-S2-C_GTCCTTCG-CTGTGCAT`
An object of class Seurat 
45282 features across 2099 samples within 2 assays 
Active assay: SCT (12578 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: Spatial
 2 dimensional reductions calculated: pca, umap
 1 image present: D14_Mid_1

$`Sam

In [87]:
# Larouche: save canine data.
for (idx in seq(1:length(ca.seurs))) {
    ca.seur <- ca.seurs[[idx]]
    ca.label <- paste0("ca", ca.labels[[idx]])
    out.dir <- file.path(laro.dir, ca.label)
    seurat_to_10x(ca.seur, out.dir)
}

In [22]:
# Chen: Human neural stem cells restore spatial memory in a transgenic Alzheimer's disease mouse model 
# by an immunomodulating mechanism
chen.dir <- file.path(raw.dir, "pub8_chen")
chen.paths <- list.files(chen.dir)
chen.path.raw <- chen.paths[1]
chen.label <- gsub("Sample_3792-FM-", "", chen.path.raw)
chen.path <- file.path(chen.dir, chen.path.raw)
chen.path

[1] "/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub8_chen/Sample_3792-FM-S2-A"

In [36]:
chen.filt.path <- file.path(chen.path, "filtered_feature_bc_matrix")
filt.mat <- Read10X(chen.filt.path)
filt.mat[1:5, 1:5]

5 x 5 sparse Matrix of class "dgCMatrix"
        AAACAAGTATCTCCCA-1 AAACAGAGCGACTCCT-1 AAACATTTCCCGGATT-1
Xkr4                     .                  .                  .
Gm1992                   .                  .                  .
Gm19938                  .                  .                  .
Gm37381                  .                  .                  .
Rp1                      .                  .                  .
        AAACCCGAACGAAATC-1 AAACCGGGTAGGTACC-1
Xkr4                     .                  .
Gm1992                   .                  .
Gm19938                  .                  .
Gm37381                  .                  .
Rp1                      .                  .

In [53]:
h5.out.path <- file.path(chen.path, "filtered_feature_bc_matrix.h5")
suppressMessages(suppressWarnings(
    write10xCounts(path=h5.out.path, x=filt.mat, type="HDF5", version="3", genome="mm10", overwrite=TRUE, 
        barcodes=colnames(filt.mat), gene.symbol=rownames(filt.mat), gene.id=rownames(filt.mat))
))
# Create_10X_H5(raw_data_file_path=chen.filt.path, save_file_path=h5.out.path, save_name=chen.label)

In [52]:
h5.out.path

[1] "/scratch/gpfs/KANG/sereno/spatialstem/sourcefiles/raw/pub8_chen/Sample_3792-FM-S2-A/filtered_feature_bc_matrix.h5"